In [1]:
import time, copy, os, sys, vtk, scipy, math, csv
from pprint import pprint
from stl import mesh
import numpy as np
np.set_printoptions(threshold=np.inf)
from scipy.special import sph_harm
from scipy import special
import scipy
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import matplotlib.tri as tri
plt.style.use('tableau-colorblind10')
plt.rcParams['text.usetex'] = True
from IPython.display import set_matplotlib_formats
import matplotlib.tri as mtri
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rc
from matplotlib import cm, colors
from matplotlib.colors import LightSource
set_matplotlib_formats('pdf') # For vectorized output of the figures
from matplotlib.ticker import LinearLocator

import pyvista as pv
from scipy.special import *

import warnings
warnings.filterwarnings("ignore")

/tmp/ipykernel_85129/355604503.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('pdf') # For vectorized output of the figures


In [2]:

def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])
    
def set_axes_equal_2d(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim()
    y_limits = ax.get_ylim()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range])

    ax.set_xlim([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim([y_middle - plot_radius, y_middle + plot_radius])


## Prolate hemispheroidal (northpole) coordinate (parametric form)


$$ x = a \sqrt{(\xi^2 - 1)(1-\eta^2)} \cos \phi $$
$$ y = a \sqrt{(\xi^2 - 1)(1-\eta^2)} \sin \phi $$
$$ z = a \xi \eta $$
with $\xi = \cosh \mu$ and $\eta = \cos \nu$


Or

$$ x = a \sinh \mu \sin \nu \cos \phi $$
$$ y = a \sinh \mu \sin \nu \sin \phi $$
$$ z = a \cosh \mu \cos \nu $$

Range of $0 \leq \mu < \infty$, $0 \leq \nu < \pi/2$ and $0 \leq \phi \leq 2\pi$

In [3]:
subdivgrid = 90
#mu = np.linspace(0, 10, num=subdivgrid)
nu = np.linspace(0, 0.5 * np.pi - 0.009, num=subdivgrid)
phi = np.linspace(0, 2*np.pi - 0.009, num=subdivgrid)
nu, phi = np.meshgrid(nu, phi)

mu = 0.3
a = 30;

x = a * np.sinh(mu) * np.sin(nu) * np.cos(phi)
y = a * np.sinh(mu) * np.sin(nu) * np.sin(phi)
z = a * np.cosh(mu) * np.cos(nu)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
r = np.sqrt(x**2 + y**2 + z**2)
scamap = plt.cm.ScalarMappable(cmap=cm.coolwarm)
fcolors = scamap.to_rgba(r)
surf = ax.plot_surface(x, y, z, facecolors = fcolors, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title("Prolate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)
plt.show()

print("Max vs min radii: ", r.max(), r.min())

<Figure size 640x480 with 2 Axes>

Max vs min radii:  31.360155423865812 9.139597706723295


In [4]:
subdivgrid = 100
#mu = np.linspace(0, 10, num=subdivgrid)
nu = np.linspace(0, 0.5 * np.pi, num=subdivgrid)
phi = np.linspace(0, 2*np.pi, num=subdivgrid)
nu, phi = np.meshgrid(nu, phi)

m, n = 5, 10

mu = 0.5
a = 10;

x = a * np.sinh(mu) * np.sin(nu) * np.cos(phi)
y = a * np.sinh(mu) * np.sin(nu) * np.sin(phi)
z = a * np.cosh(mu) * np.cos(nu)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
#r = np.sqrt(x**2 + y**2 + z**2)
r = np.exp(-1j * m * phi) * scipy.special.lpmv(m, n, (-2*np.cos(nu)+1))
r = r.real
scamap = plt.cm.ScalarMappable(cmap=cm.coolwarm)
fcolors = scamap.to_rgba(r)
surf = ax.plot_surface(x, y, z, facecolors = fcolors, cmap=cm.coolwarm,
                       linewidth=0.1, edgecolor="black", antialiased=False)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title("Prolate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)
plt.show()

print("Max vs min radii: ", r.max(), r.min())
print(np.sinh(mu), np.cosh(mu))

<Figure size 640x480 with 2 Axes>

Max vs min radii:  40440.09300203228 -40419.733129185704
0.5210953054937474 1.1276259652063807


In [5]:
subdivgrid = 30
#mu = np.linspace(0, 10, num=subdivgrid)
nu = np.linspace(0, 0.5*np.pi - 0.009, num=subdivgrid)
phi = np.linspace(0, 1.7*np.pi - 0.009, num=subdivgrid)
nu, phi = np.meshgrid(nu, phi)

mu = 0.3
a = 30;

x = a * np.sinh(mu) * np.sin(nu) * np.cos(phi)
y = a * np.sinh(mu) * np.sin(nu) * np.sin(phi)
z = a * np.cosh(mu) * np.cos(nu)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
surf = ax.plot_surface(x, y, z, cmap=plt.cm.gray,
                       linewidth=0.1, antialiased=False, edgecolors='k')
plt.title("Prolate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)
plt.show()

print("Max vs min radii: ", r.max(), r.min())

<Figure size 640x480 with 1 Axes>

Max vs min radii:  40440.09300203228 -40419.733129185704


## Oblate Spheroidal Coordinate (parametric form)


$$ x = a \sqrt{(\xi^2 - 1)(1-\eta^2)} \cos \phi $$
$$ y = a \sqrt{(\xi^2 - 1)(1-\eta^2)} \sin \phi $$
$$ z = a \xi \eta $$
with $\xi = \cosh \mu$ and $\eta = \cos \nu$


Or

$$ x = a \cosh \mu \cos \nu \cos \phi $$
$$ y = a \cosh \mu \cos \nu \sin \phi $$
$$ z = a \sinh \mu \sin \nu $$

Range of $0 \leq \mu < \infty$, $-\frac{\pi}{2} \leq \nu < \frac{\pi}{2}$ and $0 \leq \phi \leq 2\pi$

In [6]:
subdivgrid = 30
nu = np.linspace(0, np.pi/2, num=subdivgrid)
phi = np.linspace(0, 1.7*np.pi - 0.009, num=subdivgrid)
nu, phi = np.meshgrid(nu, phi)

mu = 0.3
a = 1.2;

x = a * np.cosh(mu) * np.cos(nu) * np.cos(phi)
y = a * np.cosh(mu) * np.cos(nu) * np.sin(phi)
z = a * np.sinh(mu) * np.sin(nu)


fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
surf = ax.plot_surface(x, y, z, cmap=plt.cm.gray,
                       linewidth=0.1, antialiased=False, edgecolors='k')
plt.title("Oblate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)
plt.show()

print("Max vs min radii: ", r.max(), r.min())

<Figure size 640x480 with 1 Axes>

Max vs min radii:  40440.09300203228 -40419.733129185704


In [7]:
subdivgrid = 50
nu = np.linspace(0.009, np.pi/2, num=subdivgrid)
phi = np.linspace(0, 2.0*np.pi, num=subdivgrid)
nu, phi = np.meshgrid(nu, phi)

mu = 0.5
a = 10;

m, n = 2, 5

x = a * np.cosh(mu) * np.cos(nu) * np.cos(phi)
y = a * np.cosh(mu) * np.cos(nu) * np.sin(phi)
z = a * np.sinh(mu) * np.sin(nu)


fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
r = np.exp(-1j * m * phi) * scipy.special.lpmv(m, n, (1-2*np.sin(nu)))
r = r.real
scamap = plt.cm.ScalarMappable(cmap=cm.coolwarm)
fcolors = scamap.to_rgba(r)
surf = ax.plot_surface(x, y, z, facecolors = fcolors, cmap=cm.coolwarm,
                       linewidth=0.1, antialiased=False, edgecolor="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.title("Oblate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)
plt.show()

print("Max vs min radii: ", r.max(), r.min())
print(a * np.cosh(mu), a * np.sinh(mu))
print(phi.shape)

<Figure size 640x480 with 2 Axes>

Max vs min radii:  14.436296792624162 -14.466018779682669
11.276259652063807 5.210953054937474
(50, 50)


# The elliptic cylindrical coordinates

In [8]:
subdivgrid = 30
nu = np.linspace(0, np.pi/2, num=subdivgrid)
phi = np.linspace(np.pi/2, 1.7*np.pi - 0.009, num=subdivgrid)
nu, phi = np.meshgrid(nu, phi)

mu = 0.42857142857142855
a = 1.0;

x = a * np.cosh(mu) * np.cos(nu) * np.cos(phi)
y = a * np.cosh(mu) * np.cos(nu) * np.sin(phi)
z = a * np.sinh(mu) * np.sin(nu)


fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
surf = ax.plot_surface(x, y, z, cmap=plt.cm.gray,
                       linewidth=0.1, antialiased=False, edgecolors='darkblue')
plt.title("Oblate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)





nu = np.linspace(0, np.pi/2, num=subdivgrid)
mu = np.linspace(0., 1, num=8)

a = 1.0;

# Ellipses
for i in mu:
    print(i)
    y = a * np.cosh(i) * np.cos(nu)
    z = a * np.sinh(i) * np.sin(nu)
    #plt.plot(y, z, -y, z, color="black")
    plt.plot(np.zeros(y.shape), y, z, color="black", linewidth=0.6)

nu = np.linspace(0, np.pi/2, num=8)
mu = np.linspace(0., 1.0, num=subdivgrid)
# Hyperboloids
for i in nu:
    print(i)
    y = a * np.cosh(mu) * np.cos(i)
    z = a * np.sinh(mu) * np.sin(i)
    #plt.plot(y, z, "--", -y, z, "--", color="red")
    plt.plot(np.zeros(y.shape), y, z, "--", color="red", linewidth=0.6)

plt.axis("equal")
plt.axis("off")
plt.show()

0.0
0.14285714285714285
0.2857142857142857
0.42857142857142855
0.5714285714285714
0.7142857142857142
0.8571428571428571
1.0
0.0
0.2243994752564138
0.4487989505128276
0.6731984257692414
0.8975979010256552
1.121997376282069
1.3463968515384828
1.5707963267948966


<Figure size 640x480 with 1 Axes>

In [30]:
subdivgrid = 40
nu = np.linspace(0, np.pi/2, num=subdivgrid)
mu = np.linspace(0., 1, num=8)

a = 1.0;

# Ellipses
for i in mu:
    print(i)
    y = a * np.cosh(i) * np.cos(nu)
    z = a * np.sinh(i) * np.sin(nu)
    #plt.plot(y, z, -y, z, color="black")
    plt.plot(y, z, color="black")

nu = np.linspace(0, np.pi/2, num=12)
mu = np.linspace(0., 1.0, num=subdivgrid)
# Hyperboloids
for i in nu:
    y = a * np.cosh(mu) * np.cos(i)
    z = a * np.sinh(mu) * np.sin(i)
    #plt.plot(y, z, "--", -y, z, "--", color="red")
    plt.plot(y, z, "--", color="red")

plt.axis("equal")
plt.show()

0.0
0.14285714285714285
0.2857142857142857
0.42857142857142855
0.5714285714285714
0.7142857142857142
0.8571428571428571
1.0


<Figure size 640x480 with 1 Axes>

In [10]:
subdivgrid = 30

nu = np.linspace(0, 0.5*np.pi - 0.009, num=subdivgrid)
phi = np.linspace(0, 1.7*np.pi - 0.009, num=subdivgrid)
phi = np.linspace(np.pi/2, 1.7*np.pi - 0.009, num=subdivgrid)

nu, phi = np.meshgrid(nu, phi)

mu = 0.42857142857142855
a = 1;

x = a * np.sinh(mu) * np.sin(nu) * np.cos(phi)
y = a * np.sinh(mu) * np.sin(nu) * np.sin(phi)
z = a * np.cosh(mu) * np.cos(nu)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# Plot the surface.
surf = ax.plot_surface(x, y, z, cmap=plt.cm.gray,
                       linewidth=0.1, antialiased=False, edgecolors='darkblue')
plt.title("Prolate spheroid")
#ax.set_aspect("auto")
set_axes_equal(ax)

nu = np.linspace(0, np.pi/2, num=subdivgrid)
phi = np.linspace(0, 2*np.pi - 0.009, num=subdivgrid)
mu = np.linspace(0., 1.0, num=8)

a = 1.0;


for i in mu:
    print(i)
    y = a * np.sinh(i) * np.sin(nu)
    z = a * np.cosh(i) * np.cos(nu)
    #plt.plot(y, z, y, -z, color="black")
    plt.plot(np.zeros(y.shape), y, z, color="black", linewidth=0.6)


subdivgrid = 40
nu = np.linspace(0, np.pi/2, num=8)
mu = np.linspace(0, 1.0, num=subdivgrid)

a = 1.0;

for i in nu:
    y = a * np.sinh(mu) * np.sin(i)
    z = a * np.cosh(mu) * np.cos(i)
    #plt.plot(y, z, '--', y, -z, '--', color="red")
    plt.plot(np.zeros(y.shape), y, z, '--', color="red", linewidth=0.6)
plt.axis("equal")
plt.axis("off")
plt.show()

0.0
0.14285714285714285
0.2857142857142857
0.42857142857142855
0.5714285714285714
0.7142857142857142
0.8571428571428571
1.0


<Figure size 640x480 with 1 Axes>

In [11]:
subdivgrid = 40
nu = np.linspace(0, np.pi/2, num=subdivgrid)
phi = np.linspace(0, 2*np.pi - 0.009, num=subdivgrid)
mu = np.linspace(0., 2, num=8)

a = 1.0;

fig, ax = plt.subplots()

for i in mu:
    y = a * np.sinh(i) * np.sin(nu)
    z = a * np.cosh(i) * np.cos(nu)
    #plt.plot(y, z, y, -z, color="black")
    plt.plot(y, z, color="black")


subdivgrid = 40
nu = np.linspace(0, np.pi/2, num=8)
mu = np.linspace(0, 2, num=subdivgrid)

a = 1.0;

for i in nu:
    y = a * np.sinh(mu) * np.sin(i)
    z = a * np.cosh(mu) * np.cos(i)
    #plt.plot(y, z, '--', y, -z, '--', color="red")
    plt.plot(y, z, '--', color="red")
plt.axis("equal")
plt.show()

<Figure size 640x480 with 1 Axes>

\begin{equation}
    N_m^n = \sqrt{\frac{(2n+1)}{2\pi} \ \frac{(n-m)!}{(n+m)!} \ \frac{1}{|b - a|}}.
\end{equation}

In [12]:
a, b = -1, 1
subdiv = 1000
x = np.linspace(a, b, subdiv)

n = 5
for m in range (n):
    N_mn = np.sqrt( ((2*n + 1) / abs(b - a)) * factorial(n - m) / factorial(n + m) )
    y = N_mn * scipy.special.lpmv(m, n, x)
    plt.plot(x, y, label="m = {}".format(m))
plt.title("For n = {}".format(n))
plt.legend()

<Figure size 640x480 with 1 Axes>

In [13]:
a, b = 0, 1
subdiv = 1000
x = np.linspace(a, b, subdiv)

n = 5
for m in range (n):
    N_mn = np.sqrt( ((2*n + 1) / abs(b - a)) * factorial(n - m) / factorial(n + m) )
    y = N_mn * scipy.special.lpmv(m, n, 2*x-1)
    plt.plot(x, y, label="m = {}".format(m))
plt.title("For n = {}".format(n))
plt.legend()

<Figure size 640x480 with 1 Axes>

In [14]:
a, b = 1, 0
subdiv = 1000
x = np.linspace(a, b, subdiv)

n = 5
for m in range (n):
    N_mn = np.sqrt( ((2*n + 1) / abs(b - a)) * factorial(n - m) / factorial(n + m) )
    y = N_mn * scipy.special.lpmv(m, n, -2*x + 1)
    plt.plot(x, y, label="m = {}".format(m))
plt.title("For n = {}".format(n))
plt.legend()

<Figure size 640x480 with 1 Axes>

In [15]:
def prolate_comb_func(comb = [0, 0, 1], a = 30, zeta = 0.3, subdivgrid = 50):
    '''
    comb = [[n1, m1, w1],
            [n2, m2, w2]]
    '''
    a_, b_ = 1, 0
    nu = np.linspace(0, 0.5 * np.pi - 0.009, num=subdivgrid)
    phi = np.linspace(0, 2*np.pi - 0.009, num=subdivgrid)
    nu, phi = np.meshgrid(nu, phi)
    x = a * np.sinh(zeta) * np.sin(nu) * np.cos(phi)
    y = a * np.sinh(zeta) * np.sin(nu) * np.sin(phi)
    z = a * np.cosh(zeta) * np.cos(nu)
    r = np.zeros(x.shape, dtype='complex128')
    for i in range(comb.shape[0]):
        n = comb[i, 0]
        m = comb[i, 1]
        w = comb[i, 2]
        #print("n={}, m={}, w={}".format( n, m , w))
        N_mn_ = np.sqrt( ((2*n + 1) / abs(b_ - a_)) * factorial(n - abs(m)) / (2*np.pi * factorial(n + abs(m))) )
        r += w * N_mn_ * np.exp(-1j * m * phi) * scipy.special.lpmv(m, n, (-2*np.cos(nu)+1))
    r = r.real
    
    x+=r
    y+=r
    z+=r
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    scamap = plt.cm.ScalarMappable(cmap=cm.coolwarm)
    fcolors = scamap.to_rgba(r)
    surf = ax.plot_surface(x, y, z, facecolors = fcolors, cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.title("Prolate surface")
    set_axes_equal(ax)
    plt.show()
    return x, y, z, r



comb = np.array([[0, 1, 1],
                 [2, -1, 3],
                 [3, 2, 0],
                 [5, 0, 0.5],
                 [10, 0, 0.2]
                ])
a, zeta = 30, 0.5
subdivgrid = 50
x_, y_, z_, r_ = prolate_comb_func(comb, a, zeta, subdivgrid)

<Figure size 640x480 with 2 Axes>

# Test orthonormality of the basis functions

In [16]:
n1, m1 = 4, 1
n2, m2 = 2, 1

x = np.linspace(-1, 1, 1000)

N_mn_1 = np.sqrt( ((2*n1 + 1) / 2) * factorial(n1 - abs(m1)) / (factorial(n1 + abs(m1))) )
N_mn_2 = np.sqrt( ((2*n2 + 1) / 2) * factorial(n2 - abs(m2)) / (factorial(n2 + abs(m2))) )

#P_1 = N_mn_1 * scipy.special.lpmv(m1, n1, (1*np.cos(x)+0))
P_1 = N_mn_1 * scipy.special.lpmv(m1, n1, x)
#P_2 = N_mn_2 * scipy.special.lpmv(m2, n2, (1*np.cos(x)+0))
P_2 = N_mn_2 * scipy.special.lpmv(m2, n2, x)

plt.plot(x, P_1, x, P_2)
inners = np.inner(P_1, P_2) / P_1.size
#inners2 = P_1 @ P_2

print(inners)
#print(inners2)

-6.129823500531857e-06


<Figure size 640x480 with 1 Axes>

In [17]:
t = np.linspace(0, 2*np.pi, 100)

y1 = np.cos(t)
y2 = np.sin(2*t)

plt.plot(t, y1, t, y2)
inners = np.inner(y1, y2) / y1.size
inners2 = y2 @ y1  / y1.size

print(inners)
print(inners2)

-8.961138124658101e-18
-8.961138124658101e-18


<Figure size 640x480 with 1 Axes>

# Derivative of Legendre polynomial

[Source](https://en.wikipedia.org/wiki/Associated_Legendre_polynomials)

In [18]:
n, m = 6, 1

x = np.linspace(-1, 1, 1000)

#N_mn_1 = np.sqrt( ((2*n1 + 1) / 2) * factorial(n1 - abs(m1)) / (factorial(n1 + abs(m1))) )

dPdx1 = -1 * ( scipy.special.lpmv(m+1, n, x) / (1-x**2)**0.5 + m * x * scipy.special.lpmv(m, n, x)/(1-x**2))
dPdx2 = (1 / (x**2-1)) * ( np.sqrt(1-x**2) * scipy.special.lpmv(m+1, n, x) + m * x * scipy.special.lpmv(m, n, x) )
dPdx3 = (1/(x**2-1)) * ( -1*(n+m)*(n-m+1) * np.sqrt(1-x**2) * scipy.special.lpmv(m-1, n, x) - m * x * scipy.special.lpmv(m, n, x) )
dPdx4 = (1/(x**2-1)) * ( -1*(n+1) * x * scipy.special.lpmv(m, n, x) + (n-m+1) * scipy.special.lpmv(m, n+1, x) )

plt.plot(x, dPdx2, "k*",  x, dPdx3, "r--", x, dPdx4, "g^", x, dPdx1)

inners = np.inner(P_1, P_2) / P_1.size

print(inners)


-6.129823500531857e-06


<Figure size 640x480 with 1 Axes>

# Test the angles conversion

In [19]:
subdivgrid = 40
nu = np.linspace(0, np.pi/2, num=subdivgrid)
mu = np.linspace(0., 1, num=8)

a = 1.0;

# Ellipses
for i in mu:
    print(i)
    y = a * np.cosh(i) * np.cos(nu)
    z = a * np.sinh(i) * np.sin(nu)
    #plt.plot(y, z, -y, z, color="black")
    plt.plot(y, z, color="black")

nu = np.linspace(0, np.pi/2, num=8)
mu = np.linspace(0., 1.0, num=subdivgrid)
# Hyperboloids
for i in nu:
    y = a * np.cosh(mu) * np.cos(i)
    z = a * np.sinh(mu) * np.sin(i)
    #plt.plot(y, z, "--", -y, z, "--", color="red")
    plt.plot(y, z, "--", color="red")

plt.axis("equal")
plt.show()

0.0
0.14285714285714285
0.2857142857142857
0.42857142857142855
0.5714285714285714
0.7142857142857142
0.8571428571428571
1.0


<Figure size 640x480 with 1 Axes>

In [20]:
subdivgrid = 60
nu = np.linspace(0, np.pi, num=subdivgrid)
#mu = np.linspace(0., 1, num=subdivgrid)

a = 4.0
mu = 1.7;


yy = a * np.cosh(mu) * np.cos(nu)
zz = a * np.sinh(mu) * np.sin(nu)

plt.plot(yy, zz)
plt.plot(yy, -zz)
#plt.plot(-yy, zz)
#plt.plot(-yy, -zz)

plt.axis("equal")
plt.show()

<Figure size 640x480 with 1 Axes>

In [21]:
z_y = np.tanh(mu) * np.tan(nu)
nu_computed = np.arctan(z_y) - np.pi/2

yy2 = a * np.cosh(mu) * np.cos(nu_computed)
zz2 = a * np.sinh(mu) * np.sin(nu_computed)

plt.plot(yy, zz, yy2, zz2, "r*")
plt.plot(yy, -zz, yy2, -zz2, "b*")
#plt.plot(-yy, zz, -yy2, zz2, "k*")
#plt.plot(-yy, -zz, -yy2, -zz2, "c*")

pt_mu = 0.00025
pt_nu = 0.02*np.pi
print("Input Angle:", pt_nu, " rad")
pt_y, pt_z = a * np.cosh(pt_mu) * np.cos(pt_nu), a * np.sinh(pt_mu) * np.sin(pt_nu)
plt.plot(pt_y, pt_z, "c^")

plt.axis("equal")
plt.show()

Input Angle: 0.06283185307179587  rad


<Figure size 640x480 with 1 Axes>

# Projection of a point

In [22]:
#z_y = np.tanh(mu) * np.tan(nu)
#nu_computed = np.arctan(z_y) 

#nu = np.linspace(0, np.pi, num=subdivgrid)

# Ellipses
for i in np.linspace(pt_mu, 0.7, 10):
    #print(i)
    y_ = a * np.cosh(i) * np.cos(nu)
    z_ = a * np.sinh(i) * np.sin(nu)
    
    plt.plot(y_, z_, color="black")
    
    # compute pt coordinates
    #pt_nu_comp = np.arctan( (pt_z / pt_y) * (1 / np.tanh(i))  )
    #pt_nu_comp = np.arctan2( pt_z, pt_y * np.tanh(i)  )
    #pt_nu_comp = np.sign(pt_z) * np.arccos( ( np.sqrt( (pt_y + a)**2 + pt_z**2 ) -  np.sqrt( (pt_y - a)**2 + pt_z**2 )  ) / (2 * a)) # works
    #pt_nu_comp = ( ( np.arccosh( (pt_y + 1j * pt_z) / a  ) - i ) / 1j ).real # works
    pt_nu_comp = ( ( np.arccosh( (pt_y + 1j * pt_z) / a  ) ) ).imag # works with real angle
    #pt_nu_comp = -( 1j * mu - ( np.arccos( (pt_y + 1j * pt_z) / a  ) ) ).real # works (with negative sign)
    pt_y_ = a * np.cosh(i) * np.cos(pt_nu_comp)
    pt_z_ = a * np.sinh(i) * np.sin(pt_nu_comp)
    
    plt.plot(pt_y_, pt_z_, "^")

plt.plot(y_, z_, color="red")

plt.plot(pt_y, pt_z, "c^")
plt.plot([0, pt_y], [0, pt_z])



mu = np.linspace(0., pt_mu, num=subdivgrid)

# Hyperboloids
y_h = a * np.cosh(mu) * np.cos(pt_nu)
z_h = a * np.sinh(mu) * np.sin(pt_nu)
plt.plot(y_h, z_h, "--", color="red")



    
plt.axis("equal")
plt.show()

<Figure size 640x480 with 1 Axes>

# Link: https://math.stackexchange.com/questions/1529551/elliptic-coordinates-inverting-the-transformation

$$\mu = \text{arccosh}\frac{\sqrt{(x+a)^2+y^2}+\sqrt{(x-a)^2+y^2}}{2a}$$
$$\nu = \text{sign}(y)~ \text{arccos}\frac{\sqrt{(x+a)^2+y^2}-\sqrt{(x-a)^2+y^2}}{2a}$$

In [23]:
subdivgrid = 60
nu = np.linspace(0, np.pi, num=subdivgrid)
#mu = np.linspace(0., 1, num=subdivgrid)

#a = 2.0
#mu = 0.65;

scale = 1.0
yy = a * np.cosh(mu) * np.cos(nu) * scale 
zz = a * np.sinh(mu) * np.sin(nu) * scale
rad_noise = 0.2 * np.random.normal(0, 1, len(yy))
rad_noise = 0.0 * np.random.normal(0, 1, len(yy))

yy+= rad_noise
zz+= rad_noise

colors_scatter = []
for i in range(len(yy)):
    colors_scatter.append(i)
colors_scatter = np.array(colors_scatter, dtype="float")
colors_scatter/= colors_scatter.max()

#plt.plot(yy, zz)
plt.scatter(yy, zz, c=colors_scatter)
plt.axis("equal")
plt.show()

nu_computed = np.arccosh( (yy + 1j * zz)/a ).imag


yy2 = a * np.cosh(mu) * np.cos(nu_computed)
zz2 = a * np.sinh(mu) * np.sin(nu_computed)

#plt.plot(yy, zz, yy2, zz2, "r*")
plt.scatter(yy2, zz2, c=colors_scatter)
plt.plot(yy, zz,)

#plt.plot(yy, -zz, yy2, -zz2, "b*")
#plt.plot(-yy, zz, -yy2, zz2, "k*")
#plt.plot(-yy, -zz, -yy2, -zz2, "c*")


plt.axis("equal")
plt.show()


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [24]:
np.arccosh( (yy + 1j * zz)/a ).real

array([0.00000000e+00, 4.23728814e-06, 8.47457627e-06, 1.27118644e-05,
       1.69491525e-05, 2.11864407e-05, 2.54237288e-05, 2.96610169e-05,
       3.38983051e-05, 3.81355932e-05, 4.23728814e-05, 4.66101695e-05,
       5.08474576e-05, 5.50847458e-05, 5.93220339e-05, 6.35593220e-05,
       6.77966102e-05, 7.20338983e-05, 7.62711864e-05, 8.05084746e-05,
       8.47457627e-05, 8.89830508e-05, 9.32203390e-05, 9.74576271e-05,
       1.01694915e-04, 1.05932203e-04, 1.10169492e-04, 1.14406780e-04,
       1.18644068e-04, 1.22881356e-04, 1.27118644e-04, 1.31355932e-04,
       1.35593220e-04, 1.39830508e-04, 1.44067797e-04, 1.48305085e-04,
       1.52542373e-04, 1.56779661e-04, 1.61016949e-04, 1.65254237e-04,
       1.69491525e-04, 1.73728814e-04, 1.77966102e-04, 1.82203390e-04,
       1.86440678e-04, 1.90677966e-04, 1.94915254e-04, 1.99152542e-04,
       2.03389831e-04, 2.07627119e-04, 2.11864407e-04, 2.16101695e-04,
       2.20338983e-04, 2.24576271e-04, 2.28813559e-04, 2.33050847e-04,
      

In [25]:
mu.shape

(60,)

In [26]:
nu.shape

(60,)